In [ ]:
import pandas as pd
import numpy as np
from typing import Optional
from pathlib import Path

from typing import Optional, List, Callable, Any, Union, Dict
from itertools import product
from statistics import mean
from pathlib import Path
import gzip
import os

In [ ]:
criterios = 'absolute_error'
n_arvores = 400
n_atributos =  1/3 #max_features
max_niveis = 6


resultadosCenario2 = []
contagem =1

X = df_cleaned.drop('Mortalidade da população infectada(%)', axis=1)
y = df_cleaned['Mortalidade da população infectada(%)']

  #Normalizando os dados
scaler = StandardScaler()
X_normalized = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

  #Dividindo em treinamento e teste (com sorteio do tamanho do teste e treinamento)
#tam_teste = random.uniform(0.2, 0.3)
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y ,test_size=0.2, random_state=42)

  # Inicializar e treinar o modelo RF
rf = RandomForestRegressor(n_estimators=n_arvores, criterion=criterios, max_depth=max_niveis, max_features=n_atributos)
rf.fit(X_train, y_train)

  # Fazer previsões
y_pred = rf.predict(X_test)

  # Calcular métricas de avaliação
rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
mape= mean_absolute_percentage_error(y_test, y_pred)
print(rmse,mae, mape)

In [ ]:
def print_feature_importances_random_forest(random_forest_model):

    '''
    Prints the feature importances of a Random Forest model in an ordered way.
    random_forest_model -> The sklearn.ensemble.RandomForestRegressor or RandomForestClassifier trained model
    '''

    # Fetch the feature importances and feature names
    importances = random_forest_model.feature_importances_
    features = random_forest_model.feature_names_in_

    # Organize them in a dictionary
    feature_importances = {fea: imp for imp, fea in zip(importances, features)}

    # Sorts the dictionary
    feature_importances = {k: v for k, v in sorted(feature_importances.items(), key=lambda item: item[1], reverse = True)}

    # Prints the feature importances
    for k, v in feature_importances.items():
        print(f"{k} -> {v:.4f}")

In [ ]:
print_feature_importances_random_forest(rf)

In [ ]:
def print_feature_importances_shap_values(shap_values, features):

    '''
    Prints the feature importances based on SHAP values in an ordered way
    shap_values -> The SHAP values calculated from a shap.Explainer object
    features -> The name of the features, on the order presented to the explainer
    '''

    # Calculates the feature importance (mean absolute shap value) for each feature
    importances = []
    for i in range(shap_values.values.shape[1]):
        importances.append(np.mean(np.abs(shap_values.values[:, i])))

    # Calculates the normalized version
    importances_norm = softmax(importances)

    # Organize the importances and columns in a dictionary
    feature_importances = {fea: imp for imp, fea in zip(importances, features)}
    feature_importances_norm = {fea: imp for imp, fea in zip(importances_norm, features)}

    # Sorts the dictionary
    feature_importances = {k: v for k, v in sorted(feature_importances.items(), key=lambda item: item[1], reverse = True)}
    feature_importances_norm= {k: v for k, v in sorted(feature_importances_norm.items(), key=lambda item: item[1], reverse = True)}

    # Prints the feature importances
    for k, v in feature_importances.items():
        print(f"{k} -> {v:.4f} (softmax = {feature_importances_norm[k]:.4f})")


In [ ]:
print_feature_importances_shap_values(shap_values,X.columns)